MTA Exercise: Computer Acting Weird, by @adricnet
==

In which I work through one of Brad's awesome challenges on Malware Traffic Analysis for practice and the like.

Ref: http://www.malware-traffic-analysis.net/2015/02/08/index.html

Challenge is copyright 2015 malware-traffic-analysis.net, Solution here copyleft 2016 adric@adric.net


Initial questions
---

From *capinfos* we get basic stats of the pcap including:

```
Number of packets:   2720 
File size:           2122 kB
Data size:           2079 kB
Capture duration:    1483 seconds
Start time:          Sun Feb  8 13:20:00 2015
End time:            Sun Feb  8 13:44:43 2015
```

We can run SecurityOnion's *Bro IDS* install agaisnt the pcap to get some meta:

```
bro -C -r 2015-02-08-traffic-analysis-exercise.pcap
```

And can cut that meta up to get the hosts involved:

```
cat conn.log | bro-cut -d id.orig_h | sort -u
172.16.137.1
172.16.137.40
255.255.255.255
77.72.169.154
77.72.169.155

cat conn.log | bro-cut -d id.resp_h | sort -u
172.16.137.1
172.16.137.255
172.16.137.40
173.194.39.3
178.32.74.36
178.47.141.100
192.185.35.92
194.28.190.26
216.146.39.70
2.16.162.43
224.0.0.252
239.255.255.250
255.255.255.255
31.163.203.92
71.18.62.202
77.72.169.154
```

TShark could run the statistics tables we know from WireShark to get the meta, too.

```
tshark -nn -r 2015-02-08-traffic-analysis-exercise.pcap -z conv,tcp -q
```

There are only a few RFC1918 (internal) network addresses used so I suspect user Mike's system is **137.40**

Opening Wireshark and reading the decoded DHCP traffic from the first two packets we see the IP and MAC for the client: **192.168.137.40**  **08:00:2b:ef:ab:7c**

And in the following NBNS and LLMNR traffic, a hostname: **Mike-PC**


Characterising the traffic in the capture ...
---

We can walk through the conversations in Wireshark to develop an understanding of the traffic.

**TCP**

After some initial DHCP and name traffic, we see Mike-PC HTTP connect to dyndns.org and receive the external IP in use: 212.38.170.7.

"Address A","Port A","Address B","Port B","Packets","Bytes","Packets A→B","Bytes A→B","Packets A←B","Bytes A←B","Rel Start","Duration","bps A→B","bps A←B"

clientip.dyndns w/ User-Agent: Mozilla/5.0:

"172.16.137.40","49241","216.146.39.70","80","9","881","5","394","4","487","659.281121000","0.9162","3440.24","4252.28"

connection to 12101/tcp ??:

"172.16.137.40","49242","178.47.141.100","12101","3","194","3","194","0","0","659.894200000","9.0126","172.20","N/A"

Request for harveyouellet.com/TOXICOUSTIQUE/arrowu.jpg w/ User-Agent: Mozilla/5.0 (3 times?) with no intelligble response

"172.16.137.40","49243","192.185.35.92","80","9","645","5","417","4","228","684.080454000","0.7429","4490.40","2455.18"
"172.16.137.40","49244","192.185.35.92","80","9","645","5","417","4","228","684.571297000","0.7527","4431.97","2423.23"
"172.16.137.40","49245","192.185.35.92","80","9","645","5","417","4","228","685.075329000","0.7526","4432.62","2423.59"

Request for cwvancouver.com/cp/images/digits/arrowu.jpg  w/ User-Agent: Mozilla/5.0 receives 451Kb non-JPEG

"172.16.137.40","49246","71.18.62.202","80","486","461484","165","10018","321","451466","685.826405000","12.3214","6504.46","293126.79"

connection to 12103/tcp ??:

"172.16.137.40","49247","178.47.141.100","12103","3","194","3","194","0","0","696.521043000","9.0157","172.14","N/A"

non-HTTP on 80/tcp to 173.194.39.3, twice

"172.16.137.40","49248","173.194.39.3","80","6","366","4","246","2","120","698.087919000","0.3024","6508.67","3174.96"
"172.16.137.40","49249","173.194.39.3","80","6","366","4","246","2","120","733.569107000","0.3200","6150.37","3000.18"

TLS start, certificate includes: sampo@iki.fi

"172.16.137.40","49250","31.163.203.92","443","16","2643","9","1043","7","1600","736.430374000","7.0233","1188.05","1822.51"

Download CAB? from www.download.windowsupdate.com/msdownload/update/v3/static/trustedr/en/authrootstl.cab w/ User-Agent: Microsoft-CryptoAPI/6.1

"172.16.137.40","49251","2.16.162.43","80","72","62722","26","1868","46","60854","742.414674000","60.4369","247.27","8055.21"

Non-HTTP on 80/tcp:

"172.16.137.40","49252","173.194.39.3","80","6","366","4","246","2","120","753.196887000","0.3013","6531.85","3186.27"

TLS starts to 194.28.190.26:443 with gibbersh identifiers, eg CN 457jPxEfw8rSShbih3y7p6D3 and TLS runs multiple times through to packet 2720


"172.16.137.40","49253","194.28.190.26","443","15","3771","8","983","7","2788","753.348954000","3.3176","2370.35","6722.83"

TLS starts to 178.32.74.36:443 with gibbersh identifiers, eg CN 4Tw88SdvHNKgZH3boHtKG4HL and TLS runs through to packet 2600-2640

"172.16.137.40","49260","178.32.74.36","443","112","85326","39","2879","73","82447","759.895385000","219.0652","105.14","3010.87"

**UDP**

Starting at 2688 we see a significant number (60) of UDP packets totalling almost 32 KB are sent to external address 239.255.255.250.

"Address A","Port A","Address B","Port B","Packets","Bytes","Packets A→B","Bytes A→B","Packets A←B","Bytes A←B","Rel Start","Duration","bps A→B","bps A←B"
"172.16.137.40","bootpc","172.16.137.1","bootps","5","1710","5","1710","0","0","0.000000000","1200.1412","11.40","N/A"
"255.255.255.255","bootpc","172.16.137.1","bootps","6","2089","0","0","6","2089","0.011570000","1200.1413","N/A","13.93"
"172.16.137.40","58408","224.0.0.252","llmnr","2","134","2","134","0","0","0.038010000","0.0979","10950.17","N/A"
"172.16.137.40","58501","224.0.0.252","llmnr","2","134","2","134","0","0","300.072181000","0.1053","10183.82","N/A"
"172.16.137.1","ssdp","239.255.255.250","ssdp","60","31146","60","31146","0","0","565.278644000","900.0452","276.84","N/A"

**Initial characterisation**

VT has data on that domain, our jpg URL (malicious site), and the files:

https://www.virustotal.com/en/domain/harveyouellet.com/information/

A quick hexedit to fix our HO *arrowu.jpg* and now VT tells us it's Trojan.Upatre.Crypted.2 (VT 13/56). Google search shows multiple sources classify Upatre malware family as downloader. Our CW *arrowu.jpg* (gotten the easy way with Wireshark Export Objects) matches the first one. Perhaps the malware was already present and our capture shows its secondary download (TLS) and a possible DoS attack.



**Phone a friend**

Opting to consult a peer for help, we receive an eml file that Mike received with an attachment he activated. *Viper* shows us the document attachment:

```
viper 2015-02-08-traffic-analysis-exercise-email-the-user-received.eml [not stored] > email -e
[*] Email envelope:
+---------+---------------------------------------------+
| Key     | Value                                       |
+---------+---------------------------------------------+
| Subject | Your document                               |
| To      | None                                        |
| From    | John Santoro <john.santoro@dascosupply.com> |
| Cc      | None                                        |
| Bcc     | None                                        |
| Date    | Fri, 6 Feb 2015 17:07:31 +0000              |
+---------+---------------------------------------------+
viper 2015-02-08-traffic-analysis-exercise-email-the-user-received.eml [not stored] > email -f
[*] Email attachments (total: 1):
+----+---------------------+------------------------------+-----------+--------+
| ID | FileName            | Content Type                 | File Size | MD5    |
+----+---------------------+------------------------------+-----------+--------+
| 1  | document8961294.zip | application/x-zip-compressed | 196       | 1a9d39 |
|    |                     |                              |           | 436c15 |
|    |                     |                              |           | 97108f |
|    |                     |                              |           | 8baf6d |
|    |                     |                              |           | 7dc5dd |
|    |                     |                              |           | 45     |
+----+---------------------+------------------------------+-----------+--------+
[*] Email links:
viper 2015-02-08-traffic-analysis-exercise-email-the-user-received.eml [not stored]
ed] > email -o 1
[*] Switching session to document8961294.zip
[*] Session opened on /tmp/document8961294.zip
viper document8961294.zip [not stored] > info
+----------+-----------------------------------------------------------------------------------+
| Key      | Value                                                                             |
+----------+-----------------------------------------------------------------------------------+
| Name     | document8961294.zip                                                               |
| Tags     |                                                                                   |
| Path     | /tmp/document8961294.zip                                                          |
| Size     | 11444                                                                             |
| Type     | Zip archive data, at least v2.0 to extract                                        |
| Mime     | application/zip                                                                   |
| MD5      | 1a9d39436c1597108f8baf6d7dc5dd45                                                  |
| SHA1     | 79b631306b575b0fa3e96ef6d554d1203f2fe27d                                          |
| SHA256   | db09a7db2f12bff9d64f06c721df40c8d13db47b04d0948598a9c9454af38c56                  |
| SHA512   | d0436983b49bd928005b6f5f90711afa6cc10972cdbea450fbe7e7bc1157e4691a4b4795bfccc2a46 |
|          | ef4073a260cc411c664e3fd5034ec98edf0733b7c85074a                                   |
| SSdeep   | 192:r6yb6wmPkDfbTZxQUcOVxV30Ntuy7emAPy+1BZouFpFs4zdw68XxP+znc:rVnRbFxp7X6vu+emAPZ |
|          | Fr8XxGznc                                                                         |
| CRC32    | 851BCAA9                                                                          |
| Parent   |                                                                                   |
| Children |                                                                                   |
+----------+-----------------------------------------------------------------------------------+
```

VT of the document hash confirms the SCR in this zip is Upatre. Might be worth cracking that open later...

Forensics brings the malware
---

Disk forensics recovered the second stage from the infected system before reimaging, so we have that as well.
VT confirms by hash that it is Dyreza (from VirusShare?) a credential stealer often paired with Upatre in malspam.

```
viper SMowFtlw.exe [not stored] > info
+----------+-----------------------------------------------------------------------------------+
| Key      | Value                                                                             |
+----------+-----------------------------------------------------------------------------------+
| Name     | SMowFtlw.exe                                                                      |
| Tags     |                                                                                   |
| Path     | SMowFtlw.exe                                                                      |
| Size     | 565248                                                                            |
| Type     | PE32 executable (GUI) Intel 80386, for MS Windows                                 |
| Mime     | application/x-dosexec                                                             |
| MD5      | 278d5ff287ca2172b5308fe04bb20431                                                  |
| SHA1     | 882f43bea5a210889bd149cdd44613e142a6108c                                          |
| SHA256   | cee50568b3219984dddd19622a928f6ba92ac768b375c9817c5518ba263cfa6b                  |
| SHA512   | be60180bd37fbe12e044df45f6b3a0a27da868952e96a31e8fdfdc06fdae2932881f7bc43d0613c88 |
|          | d80715dde281847d3580e30b8f4a800431f94556117a46c                                   |
| SSdeep   | 12288:T/l0ZAfDKBki/JC/stONts4rYCKfxkJdMg/ohpnxp5i/4RW6eL4w1:mZe6vcCGdMB           |
| CRC32    | BFDA9DC4                                                                          |
| Parent   |                                                                                   |
| Children |                                                                                   |
+----------+-----------------------------------------------------------------------------------+
```

MalwareBytes has a good detailed write-up inlcuding OllyDbg screenshots: https://blog.malwarebytes.com/threat-analysis/2015/11/a-technical-look-at-dyreza/

Checking your work
---

Taking the long route to collect all the available evidence helped build a compelte picture and I got all of the answers correct, in about 1000x longer than was really needed. Brad shows a Wireshark filter which combined with name resolution, a Google search, and some VT and Malwr lookups get the answers and comes to the same conclusions very quickly.

Here it is with *tshark*. Awesome!

```
$ tshark -Nn -r 2015-02-08-traffic-analysis-exercise.pcap -Y http.request
 78 659.586197 172.16.137.40 -> checkip.dyndns.com HTTP 148 GET / HTTP/1.1 
 98 684.325407 172.16.137.40 -> harveyouellet.com HTTP 171 GET /TOXICOUSTIQUE/arrowu.jpg HTTP/1.1 
107 684.823414 172.16.137.40 -> harveyouellet.com HTTP 171 GET /TOXICOUSTIQUE/arrowu.jpg HTTP/1.1 
116 685.325463 172.16.137.40 -> harveyouellet.com HTTP 171 GET /TOXICOUSTIQUE/arrowu.jpg HTTP/1.1 
127 686.061891 172.16.137.40 -> cwvancouver.com HTTP 172 GET /cp/images/digits/arrowu.jpg HTTP/1.1 
686 742.551103 172.16.137.40 -> a767.g.akamai.net HTTP 356 GET /msdownload/update/v3/static/trustedr/en/authrootstl.cab HTTP/1.1 
```

